# 코드

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/github/타이타닉/data/train.csv")
test = pd.read_csv("/content/drive/MyDrive/github/타이타닉/data/test.csv")

In [ ]:
del train['PassengerId']
del test['PassengerId']

train_name = train["Name"]
test_name = test["Name"]

del train['Name']
del test['Name']

In [ ]:
from itertools import product
case_dict = {
    "Sex" :       [0, 1],
    "Age" :       [0, 1],
    "Ticket" :    [0, 1],
    "Cabin" :     [0, 1],
    "Embarked" :  [0, 1, 2], 
}
product_case = list(product(*case_dict.values()))
product_case[0:3]

[(0, 0, 0, 0, 0), (0, 0, 0, 0, 1), (0, 0, 0, 0, 2)]

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC  # SVC: Support Vector Classification
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def hp(train_data, test_data, case_list):
  train = train_data.copy()
  test = test_data.copy()
  if case_list[0] == 0 : # sex
    train['Sex'] = train['Sex'].map({'male':0,'female':1}) 
    test['Sex'] = test['Sex'].map({'male':0,'female':1})
  else :
    train_dummy = pd.get_dummies(train['Sex'], dummy_na=False) 
    test_dummy = pd.get_dummies(test['Sex'], dummy_na=False)   
    train = pd.concat([train,train_dummy],ignore_index=False, axis=1)
    test = pd.concat([test,test_dummy],ignore_index=False, axis=1)
    train = train.drop("Sex",axis=1)
    test = test.drop("Sex",axis=1)

  if case_list[1] == 0 : # age
    train['Age'] = train['Age'].interpolate()
    test['Age'] = test['Age'].interpolate()
  else :
    train['Initial'] = 0
    for i in train:
        train['Initial'] = train_name.str.extract('([A-Za-z]+)\.') # .을 기준으로 앞에 있는 부분을 추출하는 정규표현식
        
    test['Initial']=0
    for i in test:
        test['Initial'] = test_name.str.extract('([A-Za-z]+)\.')
    train['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],['Miss','Miss','Miss','Mr','Mrs','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'], inplace = True)
    test['Initial'].replace(['Rev','Col','Dr','Dona','Ms'],['Other','Other','Mr','Mr',"Miss"],inplace=True)

    train.loc[(train.Age.isnull()) & (train.Initial=='Mr'),'Age'] = 33
    train.loc[(train.Age.isnull()) & (train.Initial=='Mrs'),'Age'] = 36
    train.loc[(train.Age.isnull()) & (train.Initial=='Master'),'Age'] = 5
    train.loc[(train.Age.isnull()) & (train.Initial=='Miss'),'Age'] = 22
    train.loc[(train.Age.isnull()) & (train.Initial=='Other'),'Age'] = 46

    test.loc[(test.Age.isnull()) & (test.Initial=='Mr'),'Age'] = 33
    test.loc[(test.Age.isnull()) & (test.Initial=='Mrs'),'Age'] = 36
    test.loc[(test.Age.isnull()) & (test.Initial=='Master'),'Age'] = 5
    test.loc[(test.Age.isnull()) & (test.Initial=='Miss'),'Age'] = 22
    test.loc[(test.Age.isnull()) & (test.Initial=='Other'),'Age'] = 46

    train_dummy = pd.get_dummies(train['Initial'], dummy_na=False) 
    test_dummy = pd.get_dummies(test['Initial'], dummy_na=False)   
    train = pd.concat([train,train_dummy],ignore_index=False, axis=1)
    test = pd.concat([test,test_dummy],ignore_index=False, axis=1)
    train = train.drop("Initial",axis=1)
    test = test.drop("Initial",axis=1)
    
  if case_list[2] == 0 : # ticket
    del train['Ticket']
    del test['Ticket']
  else :
    train_dummy = pd.get_dummies(train['Ticket'], dummy_na=False) 
    test_dummy = pd.get_dummies(test['Ticket'], dummy_na=False)   
    train = pd.concat([train,train_dummy],ignore_index=False, axis=1)
    test = pd.concat([test,test_dummy],ignore_index=False, axis=1)
    train = train.drop("Ticket",axis=1)
    test = test.drop("Ticket",axis=1)

  if case_list[3] == 0 : # cabin 
    del train['Cabin']
    del test['Cabin']
  else :
    train['Cabin'].fillna(0, inplace = True)
    test['Cabin'].fillna(0, inplace = True)

    train_dummy = pd.get_dummies(train['Cabin'], dummy_na=False) 
    test_dummy = pd.get_dummies(test['Cabin'], dummy_na=False)   
    train = pd.concat([train,train_dummy],ignore_index=False, axis=1)
    test = pd.concat([test,test_dummy],ignore_index=False, axis=1)
    train = train.drop("Cabin",axis=1)
    test = test.drop("Cabin",axis=1)

  train['Embarked'].fillna('S', inplace = True)
  test['Embarked'].fillna('S', inplace = True)
  if case_list[4] == 0 : # embarked
    train = train.drop("Embarked",axis=1)
    test = test.drop("Embarked",axis=1)
  elif case_list[4] == 1 :
    train['Embarked'] = train['Embarked'].map({'S':2,'C':1,"Q":0})
    test['Embarked'] = test['Embarked'].map({'S':2,'C':1,"Q":0})

    train['Embarked'] = train['Embarked'].astype(int)
    test['Embarked'] = test['Embarked'].astype(int)
  else:
    train_dummy = pd.get_dummies(train['Embarked'], dummy_na=False) 
    test_dummy = pd.get_dummies(test['Embarked'], dummy_na=False)   
    train = pd.concat([train,train_dummy],ignore_index=False, axis=1)
    test = pd.concat([test,test_dummy],ignore_index=False, axis=1)
    train = train.drop("Embarked",axis=1)
    test = test.drop("Embarked",axis=1)

  if len(train.isna().any().value_counts()) == 2 : # 결측치 있는지 체크
    print(train.isna().any())

  #print("##")
  best_score = -1 
  best_model = ""
  for model in model_list:
    roc_score = modeling(train,test,model)
    # roc_score = 0
    if roc_score > best_score :
      best_score = roc_score
      best_model = model
  print(case_list, "-> best model :",best_model,", score :",best_score)
  return case_list,best_model, best_score

def modeling(train, test, model2):
  x_train, x_valid, y_train, y_valid = train_test_split(
      train.drop(columns=["Survived"]),
      train["Survived"],
      test_size=0.2,
      random_state= 42, 
      stratify = None, # class 비율을 train/validation 유지
      )
  model = model2 # 선언
  model.fit(x_train,y_train) # 학습
  model_pred = model.predict(x_valid) # 예측
  #print(roc_auc_score(y_valid,model_pred))
  return roc_auc_score(y_valid,model_pred)

In [ ]:
model_list = [
    LogisticRegression(),
    SGDClassifier(),
    KNeighborsClassifier(),
    SVC(random_state=42),
    DecisionTreeClassifier(random_state=42),
    RandomForestClassifier(),
    ]

best_model = ""
best_score = -1
best_case = ""
for i in product_case:
  c_l,b_m, b_s = hp(train, test, i)
  if best_score < b_s:
    best_score = b_s
    best_model = b_m
    best_case = " ".join(map(str, c_l))

print("Best Case :", best_case)
print("Best Score :", best_score)
print("Best Model :", best_model)

(0, 0, 0, 0, 0) -> best model : DecisionTreeClassifier(random_state=42) , score : 0.7974259974259975
(0, 0, 0, 0, 1) -> best model : DecisionTreeClassifier(random_state=42) , score : 0.8129343629343628
(0, 0, 0, 0, 2) -> best model : SGDClassifier() , score : 0.8023166023166023
(0, 0, 0, 1, 0) -> best model : LogisticRegression() , score : 0.8164736164736165
(0, 0, 0, 1, 1) -> best model : LogisticRegression() , score : 0.8117117117117117
(0, 0, 0, 1, 2) -> best model : DecisionTreeClassifier(random_state=42) , score : 0.8054054054054056
(0, 0, 1, 0, 0) -> best model : DecisionTreeClassifier(random_state=42) , score : 0.8172458172458174
(0, 0, 1, 0, 1) -> best model : DecisionTreeClassifier(random_state=42) , score : 0.8267696267696268
(0, 0, 1, 0, 2) -> best model : LogisticRegression() , score : 0.8204633204633205
(0, 0, 1, 1, 0) -> best model : DecisionTreeClassifier(random_state=42) , score : 0.8279922779922779
(0, 0, 1, 1, 1) -> best model : DecisionTreeClassifier(random_state=42)

# 결과

- 모델 : RandomForestClassifier()
- Public : 0.833976 / lb : 0.7546738072
- 사용 컬럼
  - Survived  
    - 타겟
  - ~PassgerId~
    - 제거
  - Pclass
  - ~Name~
    - 제거
  - Sex  
    - male -> 0 , female -> 1 변경
  - Age 
    - Mrs, Miss 등 호칭에 따른 평균 나이대로 결측치 채우기
    - 호칭을 get_dummies화 
    - 기존 호칭(initial) 컬럼 제거
  - SibSp 
  - Parch 
  - ~Ticket~
    - 제거
  - Fare 
  - ~Cabin~ 
    - 제거
  - Embarked 
    - 결측치 2개 fillna 사용해서 'S'로 채우기
    - Embarked 컬럼 get_dummies화
    - Embarked 컬럼 제거


# 하이퍼파라미터 튜닝

In [ ]:
train['Sex'] = train['Sex'].map({'male':0,'female':1}) 
test['Sex'] = test['Sex'].map({'male':0,'female':1})

train['Initial'] = 0
for i in train:
    train['Initial'] = train_name.str.extract('([A-Za-z]+)\.') # .을 기준으로 앞에 있는 부분을 추출하는 정규표현식
        
test['Initial']=0
for i in test:
    test['Initial'] = test_name.str.extract('([A-Za-z]+)\.')
train['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],['Miss','Miss','Miss','Mr','Mrs','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'], inplace = True)
test['Initial'].replace(['Rev','Col','Dr','Dona','Ms'],['Other','Other','Mr','Mr',"Miss"],inplace=True)

train.loc[(train.Age.isnull()) & (train.Initial=='Mr'),'Age'] = 33
train.loc[(train.Age.isnull()) & (train.Initial=='Mrs'),'Age'] = 36
train.loc[(train.Age.isnull()) & (train.Initial=='Master'),'Age'] = 5
train.loc[(train.Age.isnull()) & (train.Initial=='Miss'),'Age'] = 22
train.loc[(train.Age.isnull()) & (train.Initial=='Other'),'Age'] = 46

test.loc[(test.Age.isnull()) & (test.Initial=='Mr'),'Age'] = 33
test.loc[(test.Age.isnull()) & (test.Initial=='Mrs'),'Age'] = 36
test.loc[(test.Age.isnull()) & (test.Initial=='Master'),'Age'] = 5
test.loc[(test.Age.isnull()) & (test.Initial=='Miss'),'Age'] = 22
test.loc[(test.Age.isnull()) & (test.Initial=='Other'),'Age'] = 46

train_dummy = pd.get_dummies(train['Initial'], dummy_na=False) 
test_dummy = pd.get_dummies(test['Initial'], dummy_na=False)   
train = pd.concat([train,train_dummy],ignore_index=False, axis=1)
test = pd.concat([test,test_dummy],ignore_index=False, axis=1)
train = train.drop("Initial",axis=1)
test = test.drop("Initial",axis=1)

del train['Ticket']
del test['Ticket']
del train['Cabin']
del test['Cabin']
train['Embarked'].fillna('S', inplace = True)
test['Embarked'].fillna('S', inplace = True)

train_dummy = pd.get_dummies(train['Embarked'], dummy_na=False) 
test_dummy = pd.get_dummies(test['Embarked'], dummy_na=False)   
train = pd.concat([train,train_dummy],ignore_index=False, axis=1)
test = pd.concat([test,test_dummy],ignore_index=False, axis=1)
train = train.drop("Embarked",axis=1)
test = test.drop("Embarked",axis=1)

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold

param_grid = params ={
    'n_estimators':[50,100],
    'max_depth':[6,8,10,12],
    'min_samples_leaf':[8,12,18],
    'min_samples_split':[8,16,20]
}

rf = RandomForestClassifier()
cv = KFold(n_splits=6)
grid_rf = GridSearchCV(rf, param_grid, cv=cv, scoring = 'accuracy')
grid_rf.fit(train.drop(columns=["Survived"]),train["Survived"])
print('final parmas', grid_rf.best_params_)
print('best score', grid_rf.best_score_)

final parmas {'max_depth': 8, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 50}
best score 0.8338775016627366


# submission

In [ ]:
# 학습
rf_best = grid_rf.best_estimator_

test['Fare'] = test['Fare'].fillna(7.75)

# 예측
pred = rf_best.predict(test)

# 결과 제출
submission = pd.read_csv("/content/drive/MyDrive/github/타이타닉/data/submission.csv")
submission['Survived'] = pred
submission.to_csv('/content/drive/MyDrive/github/타이타닉/submission/RandomForestClassifier_태훈.csv', index = False)

# 결과 비교

- public 
  - 0.7831 -> 0.8339 : 0.0508 상승 💖
- lb 
  - 0.7582 -> 0.7546 : 0.0036 감소 💢

- 모델 : LogisticRegression 
- public : 0.7831 / lb : 0.7582
- 사용 컬럼
  - Pclass  : 특이사항 X
  - Sex : male -> 0 , female -> 1 변경
  - Age 
    - 결측치 비율 : 17% 
    - 결측치 처리 방법 : interploate(보간법) 모두 기본 셋팅으로 사용
  - SibSp : 특이사항 X 
  - Parch :  특이사항 X 
  - Fare : 특이사항 X
  - Embarked 
    - 결측치 : 2개 발견
    - dropna(subset=["Embarked"]) 사용해서 제거 
    - S -> 2 , C - > 1 , Q -> 0 변경



- 모델 : RandomForestClassifier()
- Public : 0.833976 / lb : 0.7546738072
- 사용 컬럼
  - Survived  
    - 타겟
  - ~PassgerId~
    - 제거
  - Pclass
  - ~Name~
    - 제거
  - Sex  
    - male -> 0 , female -> 1 변경
  - Age 
    - Mrs, Miss 등 호칭에 따른 평균 나이대로 결측치 채우기
    - 호칭을 get_dummies화 
    - 기존 호칭(initial) 컬럼 제거
  - SibSp 
  - Parch 
  - ~Ticket~
    - 제거
  - Fare 
  - ~Cabin~ 
    - 제거
  - Embarked 
    - 결측치 2개 fillna 사용해서 'S'로 채우기
    - Embarked 컬럼 get_dummies화
    - Embarked 컬럼 제거
